In [2]:
import numpy as np
from math import exp
from math import log
from typing import List

In [3]:
class Neuron:
    def __init__(self, value, _children=(), _op='', label=''):
        self.value = value
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children);
        self._op = _op;
        self.label = label # for visualization

    def __repr__(self):
        stringVal = f"{self.value}";
        return stringVal

    def __add__(self,other):
        other = other if isinstance(other, Neuron) else Neuron(other)
        out = Neuron(self.value + other.value, (self, other), '+')
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out

    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)

    def __mul__(self,other):
        other = other if isinstance(other, Neuron) else Neuron(other)
        out = Neuron(self.value * other.value, (self, other), '*')
        def _backward():
            self.grad += other.value * out.grad
            other.grad += self.value * out.grad
        out._backward = _backward
        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers"
        out = Neuron(self.value**other, (self, ), f'**{other}')
        def _backward():
            self.grad += other * (self.value ** (other - 1)) * out.grad
        out._backward = _backward
        return out

    def __truediv__(self, other):
        other = other if isinstance(other, Neuron) else Neuron(other)
        return self * other**-1

    def exp(self, pos=1):
        x = self.value*pos
        out = Neuron(exp(x), (self, ), 'exp')
        def _backward():
            self.grad += out.value * out.grad * pos
        out._backward = _backward
        return out
    
    def log(self):
        x = self.value
        out = Neuron(log(x), (self,), 'log')
        def _backward():
            self.grad += (1/x) * out.grad
        out._backward = _backward
        return out

    def __radd__(self, other): # other + self
        return self + other

    def __rsub__(self, other): # other - self
        return -(self - other)

    def __rmul__(self, other): # other * self
        return self * other

    def __rtruediv__(self, other): # other / self
        other = other if isinstance(other, Neuron) else Neuron(other)
        return other * self**-1
    
    def __lt__(self,other):
        if(isinstance(other,Neuron)):
            return self.value < other.value
        else:
            return self.value < other
        
    def __gt__(self,other):
        if(isinstance(other,Neuron)):
            return self.value > other.value
        else:
            return self.value > other
    
    def __le__(self,other):
        if(isinstance(other,Neuron)):
            return self.value <= other.value
        else:
            return self.value <= other
        
    def __ge__(self,other):
        if(isinstance(other,Neuron)):
            return self.value >= other.value
        else:
            return self.value >= other


    def backward(self):
        topology = []
        visited = set()
        def build_topology(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topology(child)
                topology.append(v)
        build_topology(self)

        self.grad = 1.0
        for node in reversed(topology):
            node._backward()

    def __float__(self):
        return float(self.value)
    
    def getGrad(self):
        return float(self.grad)

    def hello(self):
        print("hello")

In [ ]:
# Wrapper, propose better name?
class LayerWrapper:
    valid_loss_func = {"mse","bce","cce"}

    def __init__(self,
                 layer_neuron_width:List[int],
                 activation_function:str="linear",
                 weight_initialization:str="uniform",
                 loss_function:str="mse",
                 uniform_parameter: tuple = (0,10,0),
                 normal_parameter: tuple = (5,3,0),
                 batch_size: int    = 1,
                 learning_rate: float = 0.1,
                 ):
        
        self.learning_rate = learning_rate
        self.loss_function = loss_function
        self.layer_selector:List[Layers] = []
        self.layer_count = len(layer_neuron_width)
        self.loss = Neuron(0)
        self.batch_size = batch_size
        self.error_acumulate = Neuron(0)
        Layers(
            wrapper=self,
            layer_neuron_width=layer_neuron_width,
            activation_function=activation_function,
            weight_initialization=weight_initialization,
            uniform_parameter=uniform_parameter,
            normal_parameter=normal_parameter
        )

        self.train_class_value = LayerWrapper.neuronGenerator(np.zeros,(len(self.layer_selector[-1]._neurons)))
        self.validation_class_value = LayerWrapper.neuronGenerator(np.zeros,(len(self.layer_selector[-1]._neurons)))

    # Function, dengan retval
    @staticmethod
    def neuronGenerator(generator, *args, **kwargs):
        value = generator(*args, **kwargs)
        return np.vectorize(Neuron)(value)
    
    def loss_func(self,Y_pred,Y_target):
        if self.loss_function == "mse":
            return LayerWrapper.meanSquareError(Y_pred,Y_target)
        elif self.loss_function == "bce":
            return LayerWrapper.binaryCrossEntropy(Y_pred,Y_target)
        elif self.loss_function == "cce":
            return LayerWrapper.categoricalCrossEntropy(Y_pred,Y_target)
        
    @staticmethod
    def meanSquareError(Y_pred,Y_target):
        tempPredictClass = Y_pred._neurons
        tempTrueClass = Y_target
        subtracted = np.subtract(tempTrueClass,tempPredictClass)
        squared = np.square(subtracted)
        return np.sum(squared)/len(tempTrueClass)

    @staticmethod
    def binaryCrossEntropy(Y_pred,Y_target):
        pass
    
    @staticmethod
    def categoricalCrossEntropy(Y_pred,Y_target):
        pass

    def validation_feedforward(self):
        pass

    def fit(self,X_train,Y_train,X_val,Y_val):
        pass

    def predict(self,inputVal:List[float]):
        for x,y in zip(self.layer_selector[0],inputVal):
            x.value = y
        self.feedforward()
        return self.getPredResult()
    
    def getPredResult(self):
        return self.layer_selector[-1]

    def __getitem__(self, idx):
        return self.layer_selector[idx]
    
    def __repr__(self):
        return f"{self.layer_selector}"
    

    # Procedure, getter, setter or something else

    def feedforward(self):
        self.layer_selector[0].feedforward()

    def setTrueClassValue(self,Y:List):
        self.train_class_value = LayerWrapper.neuronGenerator(np.array,(Y))
        print(self.train_class_value)
        self.layer_selector[-1].setLayerWidth(len(Y[0]))

    def setValidationClassValue(self,Y):
        self.validation_class_value = LayerWrapper.neuronGenerator(np.array,(Y))
        self.layer_selector[-1].setLayerWidth(len(Y[0]))

    def batchHelper(self,
                    batch_input: np.ndarray,
                    batch_class: np.ndarray,
                    current_iter=0,
                    ):
        while len(batch_input) > 0:
            if current_iter < self.batch_size:
                self.predict(batch_input[0])
                self.error_acumulate += self.loss_func(self.getPredResult(), batch_class[0])
                batch_input = batch_input[1:]
                batch_class = batch_class[1:]
                current_iter += 1
            else:
                self.backpropagationLearn()
                self.updateWeightHelper()
                current_iter = 0
            retval = self.error_acumulate
            self.error_acumulate = Neuron(0)
            return retval

    def backpropagationLearn(self):
        self.error_acumulate.backward()
        pass

    def updateWeightHelper(self):
        self.layer_selector[0].updateWeightRecursive()
        self.error_acumulate = Neuron(0)

    # Kebodohan: backward untuk grad hanya boleh dipanggil 1x dari 1 titik, kalau kyk
    # gini dipanggil berkali kali dan akhirnya refer ke node yg terakhir di layer 
    # terakhir
    # *ada alasan kenapa yg dibackprop cuma errornya doang
    # def backpropagation(self):
    #     backprop = np.vectorize(lambda x: x.backward())
    #     backprop(self.layer_selector[-1]._neurons)
    # Tidak dihapus untuk catatan kebodohan


class Layers:

    # Static Attribute, cuma buat validasi input
    valid_activation_function = {"linear","relu","sigmoid","hyperbolic_tangent","softmax","leaky_relu","swish"}
    valid_weight_initialization = {"zero","uniform","normal"}

    # START OF INITIALIZATION
    def __init__(self,
                 wrapper:LayerWrapper,                      # Wrapper ini buat ngepoint ke wrappernya
                 layer_neuron_width:list,                   # List of neuron width, elemen index ke-n menentukan lebar layer ke-n
                 activation_function:str = "linear",        # Harus ada di atas, kubuat defaultnya ini, dan rekursif layer selanjutnya sama
                 weight_initialization:str = "uniform",     # Sama kyk activation function
                 current_iter:int = 0,                      # jgn disentuh, ini buat nandain iterasinya, krn inisialisasi rekursif
                 uniform_parameter: tuple = (0,10,0),       # Hanya terpakai kalau activation_functionnya uniform
                 normal_parameter: tuple = (5,3,0),         # kyk di atas, tp normal
                 
                 # Tambah parameter kasih nama, named parameter biar ga broken pemanggilannya
                 ):
        
        # Cek validasi
        self.inputValidation(
            func_name=activation_function,
            weight_name=weight_initialization,
            layer_neuron_width=layer_neuron_width
            )
        
        self.wrapper        : LayerWrapper                      = wrapper
        self.current_iter   : int                               = current_iter
        self.neuron_count   : int                               = layer_neuron_width[current_iter]
        self.bias_weight    : Neuron                            = Neuron(0)
        self._neurons       : np.ndarray[Any, np.dtype[Neuron]] = self.neuronGenerator(np.zeros,(self.neuron_count))

        self.weight_initialization  : str    = weight_initialization
        self.uniform_parameter  : tuple = uniform_parameter
        self.normal_parameter   : tuple = normal_parameter

        self.activation_function    : str   = activation_function

        wrapper.layer_selector.append(self)
        self.next = (
            Layers(
                wrapper=wrapper,
                layer_neuron_width=layer_neuron_width,
                activation_function=activation_function,
                current_iter=current_iter + 1,
                weight_initialization=weight_initialization,
                uniform_parameter=uniform_parameter,
                normal_parameter=normal_parameter
                )
            if current_iter < len(layer_neuron_width) - 1
            else None
        )
        self.initWeight()
    # END OF INITIALIZATION

    # Buat ngeprint
    def __repr__(self):
        return f"{self._neurons}"
    
    # Kadang reflek Layer[0] dan dapet error, mending lgsg keluarin Neuron nya
    def __getitem__(self, idx):
        return self._neurons[idx]

    def __setitem__(self, idx, val):
        if (isinstance(val,Neuron)):
            self._neurons[idx] = val
        else:
            self._neurons[idx] = Neuron(val)

    # Kalau ada yg aku miss tambahin aja
    def inputValidation(self,
                        func_name:str,
                        weight_name:str,
                        layer_neuron_width:list
                        ):
        
        if func_name not in Layers.valid_activation_function:
            raise ValueError(f"Valid activation function name: {Layers.valid_activation_function}")
        
        if weight_name not in Layers.valid_weight_initialization:
            raise ValueError(f"Valid weight initialization: {Layers.valid_weight_initialization}")
        
        if len(layer_neuron_width) < 1:
            raise ValueError(f"layer_neuron_width must be a list with at least one positive integer.")
        
        if not all(isinstance(n, int) and n > 0 for n in layer_neuron_width):
            raise ValueError(f"All elements in layer_neuron_width must be a positive integer")

    # Init weightnya ngikut dari named parameter    
    def initWeight(self):
        if self.weight_initialization=="normal":
            mean        = self.normal_parameter[0]
            variance    = self.normal_parameter[1]
            seed        = self.normal_parameter[2]
            self.initWeightNormal(mean,variance,seed)

        elif self.weight_initialization=="uniform":
            low     = self.uniform_parameter[0]
            high    = self.uniform_parameter[1]
            seed    = self.uniform_parameter[2]
            self.initWeightUniform(low,high,seed)
            
        elif self.weight_initialization=="zero":
            self.initWeightZero()

    # Tiga function di bawah buat initialization, buat manual assign per layer
    def initWeightUniform(
            self,
            low     : float =   0,
            high    : float =   10,
            seed    : int   =   0
            ):
        
        if self.next != None:
            self.weight_initialization = "uniform"
            rng = np.random.default_rng(seed=seed)
            next_neuron_count = self.next.neuron_count
            self.weight = self.neuronGenerator(rng.uniform,low,high,size=(next_neuron_count,self.neuron_count))

    def initWeightNormal(
            self,
            mean        :   float   =   5,
            variance    :   float   =   3,
            seed        :   int     =   0
            ):
        if self.next != None:
            self.weight_initialization = "normal"
            rng = np.random.default_rng(seed=seed)
            next_neuron_count = self.next.neuron_count
            self.weight = self.neuronGenerator(rng.normal,loc=mean,scale=variance*variance,size=(next_neuron_count,self.neuron_count))

    def initWeightZero(
            self
        ):
        if self.next != None:
            self.weight_initialization = "zero"
            next_neuron_count = self.next.neuron_count
            self.weight = self.neuronGenerator(np.zeros,shape=(next_neuron_count,self.neuron_count))
        
    # Feedforward rekursif, biar ga tolol kyk sebelumnya, mohon maaf
    def feedforward(self):
        if self.next != None:
            # INI BABI, GW BARU NYADAR SETELAH DEBUGGING BEBERAPA JAM, 
            # HARUSNYA CUMA REPLACE VALUE DOANG, GARA GARA NEURONNYA KEREPLACE WEIGHTNYA JG ILANG
            self.next._neurons = np.dot(self._neurons,self.weight.T) + self.bias_weight
            self.next._neurons = self.activate(self.next._neurons)
            self.next.feedforward()

    # Update weight
    def updateWeightRecursive(self):
        if self.next is not None:
            self.weight = Layers.vectorUpdateWeight(self.weight, self.wrapper.learning_rate)
            self.next.updateWeightRecursive()

    # set Layer ini ngisi manual node nya, baru sadar ga guna jg sih
    # node hidden layer bakal kereplace jg pas feedforward
    def setLayerNeurons(
            self,
            neuron_list:List[float]
            ):
        self._neurons = self.neuronGenerator(np.array,(neuron_list))
        if self.current_iter>0:
            self.wrapper.layer_selector[self.current_iter-1].initWeight()
        self.initWeight()

    # Set lebar/jumlah neuron dari 1 layer
    # di bawah ada set weight karena weight depend on next hidden layer width
    def setLayerWidth(
            self,
            width: int = 1,
        ):
        self._neurons = self.neuronGenerator(np.zeros,(width))
        self.neuron_count = width
        if self.current_iter>0:
            self.wrapper.layer_selector[self.current_iter-1].initWeight()
        self.initWeight()

    def setBias(self, bias:float):
        self.bias_weight = Neuron(bias)

    # Masih sama dgn kode lama, buat fill in neuron di numpy
    def neuronGenerator(self,generator, *args, **kwargs):
        value = generator(*args, **kwargs)
        return np.vectorize(Neuron)(value)
    
    @staticmethod
    def vectorUpdateWeight(inputVal:Neuron,rate:float):
        def updateSingleNeuron(inputVal:Neuron,rate:float):
            inputVal._prev = set()
            return inputVal-(rate*inputVal.grad)
        updateTemp = np.vectorize(updateSingleNeuron)
        return updateTemp(inputVal,rate)
    
    # ACTIVATION FUNCTION

    def activate(self,input):
        if self.next.activation_function=="linear":
            return Layers.linear(input)
        elif self.next.activation_function=="relu":
            return Layers.relu(input)
        elif self.next.activation_function=="sigmoid":
            return Layers.sigmoid(input)
        elif self.next.activation_function=="hyperbolic_tangent":
            return Layers.hyperbolic_tangent(input)
        elif self.next.activation_function=="softmax":
            return Layers.soft_max(input)
    
    @staticmethod
    def linear(inputVal):
        return inputVal

    @staticmethod
    def relu(inputVal):
        reluTemp = np.vectorize(lambda x: x if x > 0 else Neuron(0))
        return reluTemp(inputVal)
    
    # TODO: add parameter, alphanya masih hardcoded 0.5
    def leaky_relu(self):
        leakytemp = np.vectorize(lambda x: x if x > 0 else Neuron(0)*Neuron(0.5))
        return leakytemp(self._neurons)
    
    # TODO: add parameter, beta nya masih hardcoded 1
    def swish(self):
        def sigmoidScalar(x:Neuron):
            return x/(1+x.exp(-1*1))
        
        vectorized_sigmoid = np.vectorize(sigmoidScalar)
        return vectorized_sigmoid(self._neurons)
    
    @staticmethod
    def sigmoid(inputVal):
        def sigmoidScalar(x:Neuron):
            return 1/(1+x.exp(-1))
        
        vectorized_sigmoid = np.vectorize(sigmoidScalar)
        return vectorized_sigmoid(inputVal)
    
    @staticmethod
    def hyperbolic_tangent(inputVal):
        def tanhScalar(x:Neuron):
            return (x.exp()-x.exp(-1))/(x.exp()+x.exp(-1))
        
        vectorized_tanh = np.vectorize(tanhScalar)
        return vectorized_tanh(inputVal)
    
    @staticmethod
    def soft_max(inputVal):
        exp = np.vectorize(lambda x: x.exp())
        divide = np.vectorize(lambda x,y: x/y)
        temp = exp(inputVal)
        sumTemp = np.sum(temp)
        return divide(temp,sumTemp)



In [30]:
xorFunc = LayerWrapper(
    activation_function="relu",
    weight_initialization="uniform",
    loss_function="mse",
    layer_neuron_width=[2,3,3,1],
    batch_size=4,
    learning_rate=0.75
)

In [31]:
# xorFunc.setTrueClassValue([[0]])

output = xorFunc.predict([1,0])
error = xorFunc.meanSquareError(output,[[1]])

# xorFunc.layer_selector[3]
# error.backward()


In [32]:
xorFunc.layer_selector[3][0]


533.9638449809597

In [33]:
error

284050.46005688846

In [34]:
error.backward()

In [35]:
xorFunc[0].weight

array([[6.369616873214543, 2.697867137638703],
       [0.4097352393619469, 0.16527635528529094],
       [8.132702392002724, 9.127555772777217]], dtype=object)

In [36]:
xorFunc.updateWeightHelper()

In [37]:
xorFunc[0].weight

array([[-34772.329966535115, 2.697867137638703],
       [-33667.71716672718, 0.16527635528529094],
       [-23545.308317555384, 9.127555772777217]], dtype=object)

In [8]:
from graphviz import Digraph

def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root, format='svg', rankdir='LR'):
    """
    format: png | svg | ...
    rankdir: TB (top to bottom graph) | LR (left to right)
    """
    assert rankdir in ['LR', 'TB']
    nodes, edges = trace(root)
    dot = Digraph(format=format, graph_attr={'rankdir': rankdir}) #, node_attr={'rankdir': 'TB'})
    
    for n in nodes:
        dot.node(name=str(id(n)), label = "{ data %.4f | grad %.4f }" % (n.value, n.grad), shape='record')
        if n._op:
            dot.node(name=str(id(n)) + n._op, label=n._op)
            dot.edge(str(id(n)) + n._op, str(id(n)))
    
    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
    
    return dot
